<a href="https://colab.research.google.com/github/SunnieCoder/Machine-Learning-projects/blob/main/K_fold_Fine_tune_BERT_base_multilingual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras
!pip install scikit-learn
!pip install transformers
!pip install torch torchvision torchaudio
!pip install transformers seqeval[gpu]
!pip install transformers datasets tokenizers seqeval -q
!pip install -U accelerate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import gzip
import csv
import pickle
import datasets
import accelerate

import torch
from torch.optim import AdamW
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from keras.preprocessing.sequence import pad_sequences
from seqeval.metrics import classification_report

import transformers
from transformers import BertTokenizer, BertConfig
from transformers import get_linear_schedule_with_warmup
from transformers import BertForTokenClassification, AdamW
from transformers import DataCollatorForTokenClassification
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import TrainingArguments, Trainer

from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
device = torch.device('cuda')
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
print(device)

cuda


In [ ]:
torch.__version__

'2.1.0+cu118'

# Load given dataset (Train_Tagged_Titles)

In [ ]:
# Load training data
data_path = "drive/MyDrive/Deep Dream/Train_Tagged_Titles.tsv"
# Read the file using Python's open to handle line terminators
with open(data_path, 'r', encoding='utf-8', newline='\r\n') as file:
    train_tagged_titles = pd.read_csv(file, sep='\t', quotechar='', quoting=3)


In [ ]:
# Show the resulting DataFrame
train_tagged_titles.head()


,Record Number,Title,Token,Tag
0,1,Supreme Nike SB Dunk High By any Means Red US1...,Supreme,Modell
1,1,Supreme Nike SB Dunk High By any Means Red US1...,Nike,Marke
2,1,Supreme Nike SB Dunk High By any Means Red US1...,SB,Produktlinie
3,1,Supreme Nike SB Dunk High By any Means Red US1...,Dunk,NaN
4,1,Supreme Nike SB Dunk High By any Means Red US1...,High,Schuhschaft-Typ


In [ ]:
one = train_tagged_titles[train_tagged_titles['Tag'] == "UK-Schuhgröße"]
one['Token'] = pd.to_numeric(one['Token'], errors='coerce')
two = one[one['Token'] > 9]
two

<ipython-input-9-63ae179509c9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one['Token'] = pd.to_numeric(one['Token'], errors='coerce')


,Record Number,Title,Token,Tag
444,38,Sportschuhe Herren Adidas Terrex Tracerocker G...,10.0,UK-Schuhgröße
4569,414,Nike Air Shake Ndestrukt / Pigalle AA4315 100 ...,10.0,UK-Schuhgröße
5363,484,DC SHOES TONIK 302905 SCHWARZ SKATESCHUHE SNEA...,11.0,UK-Schuhgröße
15976,1442,Park Authority GK 3000 Black Lucite Green 44.5...,9.5,UK-Schuhgröße
16127,1455,Adidas Samba Sneaker schwarz Retro Gr . 10 / 44,10.0,UK-Schuhgröße
19765,1787,"Gayle Nike Air Force 1 mid - heavy used , Gr ....",10.0,UK-Schuhgröße
23932,2159,adidas ZX Flux Techfit Aqua ( 2015 ) 8 9 10 11...,10.0,UK-Schuhgröße
23933,2159,adidas ZX Flux Techfit Aqua ( 2015 ) 8 9 10 11...,11.0,UK-Schuhgröße
23991,2164,Größe 12.5 Onitsuka Tiger Mexico 66 Herren Led...,12.5,UK-Schuhgröße
25793,2324,DACHSTEIN LEGGERA LC DDS OCEAN / LIME OUTDOORS...,11.0,UK-Schuhgröße


In [ ]:
# Select rows with Tag being "EU-Schuhgröße" or "US-Schuhgröße"
selected_rows = train_tagged_titles[train_tagged_titles['Tag'].isin(["EU-Schuhgröße", "UK-Schuhgröße"]) | (train_tagged_titles['Token'] == "/")]

# Group by Record Number and filter where both Tags exist for the same Record Number
res = selected_rows[selected_rows.duplicated(subset='Record Number', keep=False)]
res

,Record Number,Title,Token,Tag
56,5,"Nike air jordan 9 og , space jam , hare , bugs...",EU,EU-Schuhgröße
58,5,"Nike air jordan 9 og , space jam , hare , bugs...",/,No Tag
102,9,Adidas NMD R1 STLT PK Primeknit Sneaker Turnsc...,36,EU-Schuhgröße
104,9,Adidas NMD R1 STLT PK Primeknit Sneaker Turnsc...,42,EU-Schuhgröße
219,20,Nike Jordan 1 Low ( W ) Marina Blue | Größe 40...,"40,5",EU-Schuhgröße
...,...,...,...,...
55152,4998,Adidas Superstar Laceless / Adidas Originals /...,/,No Tag
55155,4998,Adidas Superstar Laceless / Adidas Originals /...,/,No Tag
55158,4998,Adidas Superstar Laceless / Adidas Originals /...,/,No Tag
55160,4998,Adidas Superstar Laceless / Adidas Originals /...,/,No Tag


In [ ]:
three = train_tagged_titles[(train_tagged_titles['Tag'] == "EU-Schuhgröße") | (train_tagged_titles['Tag'] == "US-Schuhgröße")]
three

,Record Number,Title,Token,Tag
9,1,Supreme Nike SB Dunk High By any Means Red US1...,US10,US-Schuhgröße
10,1,Supreme Nike SB Dunk High By any Means Red US1...,EU44,EU-Schuhgröße
33,3,♥ MICHAEL KORS Sneaker Gr 39 ♥,39,EU-Schuhgröße
56,5,"Nike air jordan 9 og , space jam , hare , bugs...",EU,EU-Schuhgröße
59,5,"Nike air jordan 9 og , space jam , hare , bugs...",US,US-Schuhgröße
...,...,...,...,...
55090,4991,DAMEN H750N 7272 GEL-LYTE KOMACHI PEACH BEIGE ...,38,EU-Schuhgröße
55093,4992,Sneaker 38 beige sand Tamaris Schnürung Textil...,38,EU-Schuhgröße
55109,4993,WODEN Sneaker Ydun Croco Echtleder neu Gr . 37,37,EU-Schuhgröße
55115,4994,Lonsdale Camden Herren Leder Schuhe 41 42 43 4...,41,EU-Schuhgröße


In [ ]:
# The dataset contains 55,184 data records and 1 header record.
num_records = train_tagged_titles.shape[0]+1

assert num_records == 55184

# Create new dataframe df where each row is 1 unique title, aggregate tokens into Token List and aggregate Tag List

In [ ]:
df_token = train_tagged_titles.groupby('Record Number')['Token'].agg(list).reset_index()
df_tag = train_tagged_titles.groupby('Record Number')['Tag'].agg(list).reset_index()
df_title = train_tagged_titles.groupby('Record Number')['Title'].first().reset_index()

# Merging both dataframes on 'Record Number'
df = pd.merge(df_title, df_token, on='Record Number').merge(df_tag, on='Record Number')
df = df.rename(columns={'Title': 'Title', 'Token': 'Token List', 'Tag': 'Tag List'})


In [ ]:
df.head()

,Record Number,Title,Token List,Tag List
0,1,Supreme Nike SB Dunk High By any Means Red US1...,"[Supreme, Nike, SB, Dunk, High, By, any, Means...","[Modell, Marke, Produktlinie, nan, Schuhschaft..."
1,2,New Balance 530 Männer und Frauen Laufschuhe m...,"[New, Balance, 530, Männer, und, Frauen, Laufs...","[Marke, nan, Modell, Abteilung, No Tag, Abteil..."
2,3,♥ MICHAEL KORS Sneaker Gr 39 ♥,"[♥, MICHAEL, KORS, Sneaker, Gr, 39, ♥]","[No Tag, Marke, nan, Stil, No Tag, EU-Schuhgrö..."
3,4,New Balance ML 574 EGO Turnschuhe grün,"[New, Balance, ML, 574, EGO, Turnschuhe, grün]","[Marke, nan, Modell, nan, nan, Produktart, Farbe]"
4,5,"Nike air jordan 9 og , space jam , hare , bugs...","[Nike, air, jordan, 9, og, ,, space, jam, ,, h...","[Marke, Produktlinie, nan, Modell, nan, No Tag..."


In [ ]:
df.tail()

,Record Number,Title,Token List,Tag List
4995,4996,Adidas Raf Simons Sneakers,"[Adidas, Raf, Simons, Sneakers]","[Marke, Produktlinie, nan, Stil]"
4996,4997,"Ludwig Reiter Andalusier , Nubuk navy Schuhe D...","[Ludwig, Reiter, Andalusier, ,, Nubuk, navy, S...","[Marke, nan, Modell, No Tag, Obermaterial, Far..."
4997,4998,Adidas Superstar Laceless / Adidas Originals /...,"[Adidas, Superstar, Laceless, /, Adidas, Origi...","[Marke, Produktlinie, Verschluss, No Tag, Mark..."
4998,4999,HUGO BOSS Herren Schuhe Schnürschuhe Sneaker V...,"[HUGO, BOSS, Herren, Schuhe, Schnürschuhe, Sne...","[Marke, nan, Abteilung, Produktart, nan, Stil,..."
4999,5000,Herren Trekking Schuhe Outdoor Sneaker Sportsc...,"[Herren, Trekking, Schuhe, Outdoor, Sneaker, S...","[Abteilung, Aktivität, Produktart, Anlass, Sti..."


In [ ]:
# Find the longest token list
TokenList = list(df['Token List'].values)
longestToken = max(TokenList, key = len)
longest_title = df.loc[df['Token List'].apply(len).idxmax(), 'Title']
print("Longest tile is:", longest_title)
print("The tokens in the longest tile is:", longestToken)
print("Then length of this token is:", len(longestToken))

Longest tile is: Asics Gel-Lyte III " Holiday Pack " ( HL6A2-2121 ) 44 45 46 ( GL / 3 / V / KITH / OG / Saga / GT )
The tokens in the longest tile is: ['Asics', 'Gel-Lyte', 'III', '"', 'Holiday', 'Pack', '"', '(', 'HL6A2-2121', ')', '44', '45', '46', '(', 'GL', '/', '3', '/', 'V', '/', 'KITH', '/', 'OG', '/', 'Saga', '/', 'GT', ')']
Then length of this token is: 28


**We need to self generate the token because the Listing_Titles file only has record number and title columns.

The below code is to verify if Self-generated token is correct.

The result indicate that our Self-generated token is correct.**

In [ ]:
# Function to split the title into a list of strings
def split_title(title):
    return title.split()

# Apply the function to create Self-generated token list
df['Self-generated Token List'] = df['Title'].apply(lambda x: split_title(x))
df = df[['Record Number', 'Title', 'Token List', 'Self-generated Token List', 'Tag List']]
df.head()

,Record Number,Title,Token List,Self-generated Token List,Tag List
0,1,Supreme Nike SB Dunk High By any Means Red US1...,"[Supreme, Nike, SB, Dunk, High, By, any, Means...","[Supreme, Nike, SB, Dunk, High, By, any, Means...","[Modell, Marke, Produktlinie, nan, Schuhschaft..."
1,2,New Balance 530 Männer und Frauen Laufschuhe m...,"[New, Balance, 530, Männer, und, Frauen, Laufs...","[New, Balance, 530, Männer, und, Frauen, Laufs...","[Marke, nan, Modell, Abteilung, No Tag, Abteil..."
2,3,♥ MICHAEL KORS Sneaker Gr 39 ♥,"[♥, MICHAEL, KORS, Sneaker, Gr, 39, ♥]","[♥, MICHAEL, KORS, Sneaker, Gr, 39, ♥]","[No Tag, Marke, nan, Stil, No Tag, EU-Schuhgrö..."
3,4,New Balance ML 574 EGO Turnschuhe grün,"[New, Balance, ML, 574, EGO, Turnschuhe, grün]","[New, Balance, ML, 574, EGO, Turnschuhe, grün]","[Marke, nan, Modell, nan, nan, Produktart, Farbe]"
4,5,"Nike air jordan 9 og , space jam , hare , bugs...","[Nike, air, jordan, 9, og, ,, space, jam, ,, h...","[Nike, air, jordan, 9, og, ,, space, jam, ,, h...","[Marke, Produktlinie, nan, Modell, nan, No Tag..."


In [ ]:
# Check if the tokens in the 'Self-generated Token List' is the same as 'Token List'
df_check = df.copy()
df_check['Token Match'] = df_check.apply(lambda row: row['Self-generated Token List'] == row['Token List'], axis=1)
df_check.head()

# Check if there are any False values in the 'Title Match' column
any_false_values = any(df_check['Token Match'] == False)

print("Any False values in 'Token Match' column:", any_false_values)

Any False values in 'Token Match' column: False


**To use BERT pretrained model, we need to convert the Tag format into IOB format.**

In [ ]:
# # Function to convert the Tag format into IOB format
# def covert_tag(tag_list):
#   newTag = tag_list.copy()
#   for i in range(len(newTag)):
#     if newTag[i] in ['No Tag', 'Obscure']:
#       newTag[i] = 'O'
#       continue
#     elif i > 0 and pd.isna(newTag[i]):
#       if newTag[i-1] == 'O':
#         newTag[i] = 'O'
#       else:
#         newTag[i] = '-'.join(['I'] + newTag[i-1].split('-')[1:])
#     else:
#       newTag[i] = '-'.join(['B', newTag[i]])
#   return newTag

# # Create a new DataFrame with the modified column
# new_df = pd.DataFrame({'IOB Tag List': df['Tag List'].apply(covert_tag)})

In [ ]:
# Function to convert the Tag format into IOB format
def covert_tag(tag_list):
  newTag = tag_list.copy()
  for i in range(len(newTag)):
    if newTag[i] == 'Obscure':
      continue
    elif newTag[i] == 'No Tag':
      newTag[i] = 'NoTag'
    elif i < len(newTag) - 1 and pd.isna(newTag[i+1]):
      if not pd.isna(newTag[i]):
        newTag[i] = '-'.join(['B', newTag[i]])
      elif i > 0 and pd.isna(newTag[i]):
        newTag[i] = '-'.join(['I'] + newTag[i-1].split('-')[1:])
    elif pd.isna(newTag[i]):
        newTag[i] = '-'.join(['L'] + newTag[i-1].split('-')[1:])
    else:
      newTag[i] = '-'.join(['U', newTag[i]])
  return newTag

# Create a new DataFrame with the modified column
new_df = pd.DataFrame({'IOB Tag List': df['Tag List'].apply(covert_tag)})

In [ ]:
# Sanity check an example

exam = df.iloc[4:5]
exam

,Record Number,Title,Token List,Self-generated Token List,Tag List
4,5,"Nike air jordan 9 og , space jam , hare , bugs...","[Nike, air, jordan, 9, og, ,, space, jam, ,, h...","[Nike, air, jordan, 9, og, ,, space, jam, ,, h...","[Marke, Produktlinie, nan, Modell, nan, No Tag..."


In [ ]:
ex = exam['Tag List'].item()
ex

['Marke',
 'Produktlinie',
 nan,
 'Modell',
 nan,
 'No Tag',
 'Farbe',
 'Modell',
 'No Tag',
 'Farbe',
 'No Tag',
 'Farbe',
 nan,
 'No Tag',
 'EU-Schuhgröße',
 nan,
 'No Tag',
 'US-Schuhgröße',
 nan,
 'No Tag',
 'No Tag',
 'No Tag']

In [ ]:
covert_tag(ex)

['U-Marke',
 'B-Produktlinie',
 'L-Produktlinie',
 'B-Modell',
 'L-Modell',
 'NoTag',
 'U-Farbe',
 'U-Modell',
 'NoTag',
 'U-Farbe',
 'NoTag',
 'B-Farbe',
 'L-Farbe',
 'NoTag',
 'B-EU-Schuhgröße',
 'L-EU-Schuhgröße',
 'NoTag',
 'B-US-Schuhgröße',
 'L-US-Schuhgröße',
 'NoTag',
 'NoTag',
 'NoTag']

In [ ]:
# Concatenate the new DataFrame with the original DataFrame
df = pd.concat([df, new_df], axis=1)
df.head()

,Record Number,Title,Token List,Self-generated Token List,Tag List,IOB Tag List
0,1,Supreme Nike SB Dunk High By any Means Red US1...,"[Supreme, Nike, SB, Dunk, High, By, any, Means...","[Supreme, Nike, SB, Dunk, High, By, any, Means...","[Modell, Marke, Produktlinie, nan, Schuhschaft...","[U-Modell, U-Marke, B-Produktlinie, L-Produktl..."
1,2,New Balance 530 Männer und Frauen Laufschuhe m...,"[New, Balance, 530, Männer, und, Frauen, Laufs...","[New, Balance, 530, Männer, und, Frauen, Laufs...","[Marke, nan, Modell, Abteilung, No Tag, Abteil...","[B-Marke, L-Marke, U-Modell, U-Abteilung, NoTa..."
2,3,♥ MICHAEL KORS Sneaker Gr 39 ♥,"[♥, MICHAEL, KORS, Sneaker, Gr, 39, ♥]","[♥, MICHAEL, KORS, Sneaker, Gr, 39, ♥]","[No Tag, Marke, nan, Stil, No Tag, EU-Schuhgrö...","[NoTag, B-Marke, L-Marke, U-Stil, NoTag, U-EU-..."
3,4,New Balance ML 574 EGO Turnschuhe grün,"[New, Balance, ML, 574, EGO, Turnschuhe, grün]","[New, Balance, ML, 574, EGO, Turnschuhe, grün]","[Marke, nan, Modell, nan, nan, Produktart, Farbe]","[B-Marke, L-Marke, B-Modell, I-Modell, L-Model..."
4,5,"Nike air jordan 9 og , space jam , hare , bugs...","[Nike, air, jordan, 9, og, ,, space, jam, ,, h...","[Nike, air, jordan, 9, og, ,, space, jam, ,, h...","[Marke, Produktlinie, nan, Modell, nan, No Tag...","[U-Marke, B-Produktlinie, L-Produktlinie, B-Mo..."


In [ ]:
# Sanity check if the IOB tag has exceptions

# Filter rows with 'I' or 'B' in the last column
filtered_rowswithI = df[df['IOB Tag List'].apply(lambda x: 'I' in x)]

# Display the filtered rows
filtered_rowswithI.shape[0]

0

In [ ]:
# Sanity check if the IOB tag has exceptions

# Filter rows with 'I' or 'B' in the last column
filtered_rowswithO = df[df['IOB Tag List'].apply(lambda x: 'B' in x)]
filtered_rowswithO.shape[0]

0

In [ ]:
# sanity check the 5th row
df['Self-generated Token List'][4]

['Nike',
 'air',
 'jordan',
 '9',
 'og',
 ',',
 'space',
 'jam',
 ',',
 'hare',
 ',',
 'bugs',
 'bunny',
 ',',
 'EU',
 '45',
 '/',
 'US',
 '11',
 'NEU',
 '+',
 'RECHNUNG']

In [ ]:
# sanity check the 5th row
print("Example title:",df['Title'][4])
print("Example original Tags:",df['Tag List'][4])
print("")
for i in range(len(df["IOB Tag List"][4])):
  print(f'{df["Self-generated Token List"][4][i]} --> {df["IOB Tag List"][4][i]}')

Example title: Nike air jordan 9 og , space jam , hare , bugs bunny , EU 45 / US 11 NEU + RECHNUNG
Example original Tags: ['Marke', 'Produktlinie', nan, 'Modell', nan, 'No Tag', 'Farbe', 'Modell', 'No Tag', 'Farbe', 'No Tag', 'Farbe', nan, 'No Tag', 'EU-Schuhgröße', nan, 'No Tag', 'US-Schuhgröße', nan, 'No Tag', 'No Tag', 'No Tag']

Nike --> U-Marke
air --> B-Produktlinie
jordan --> L-Produktlinie
9 --> B-Modell
og --> L-Modell
, --> NoTag
space --> U-Farbe
jam --> U-Modell
, --> NoTag
hare --> U-Farbe
, --> NoTag
bugs --> B-Farbe
bunny --> L-Farbe
, --> NoTag
EU --> B-EU-Schuhgröße
45 --> L-EU-Schuhgröße
/ --> NoTag
US --> B-US-Schuhgröße
11 --> L-US-Schuhgröße
NEU --> NoTag
+ --> NoTag
RECHNUNG --> NoTag


In [ ]:
# Get the Token List and IOB Tag List to use later
tokens_list = list(df['Self-generated Token List'].values)
tags_list = list(df['IOB Tag List'].values)

In [ ]:
# Sanity check
print(tokens_list[4])
print("The length of this token is", len(tokens_list[4]))
print(tags_list[4])

['Nike', 'air', 'jordan', '9', 'og', ',', 'space', 'jam', ',', 'hare', ',', 'bugs', 'bunny', ',', 'EU', '45', '/', 'US', '11', 'NEU', '+', 'RECHNUNG']
The length of this token is 22
['U-Marke', 'B-Produktlinie', 'L-Produktlinie', 'B-Modell', 'L-Modell', 'NoTag', 'U-Farbe', 'U-Modell', 'NoTag', 'U-Farbe', 'NoTag', 'B-Farbe', 'L-Farbe', 'NoTag', 'B-EU-Schuhgröße', 'L-EU-Schuhgröße', 'NoTag', 'B-US-Schuhgröße', 'L-US-Schuhgröße', 'NoTag', 'NoTag', 'NoTag']


**Set of unique tags and its indices**

In [ ]:
# Set of unique tags and its indices

flat_tags = sorted([tag for sublist in tags_list for tag in sublist])
tag_values = []

for tag in flat_tags:
    if tag not in tag_values:
        tag_values.append(tag)

# tag_values.append('PAD')

tag2idx = {t: i for i, t in enumerate(tag_values)}
idx2tag = {i: t for i, t in enumerate(tag_values)}
print(tag2idx)
print(idx2tag)

{'B-Abteilung': 0, 'B-Aktivität': 1, 'B-Akzente': 2, 'B-Anlass': 3, 'B-Besonderheiten': 4, 'B-Charakter': 5, 'B-Charakter Familie': 6, 'B-Dämpfungsgrad': 7, 'B-EU-Schuhgröße': 8, 'B-Erscheinungsjahr': 9, 'B-Farbe': 10, 'B-Futtermaterial': 11, 'B-Gewebeart': 12, 'B-Herstellernummer': 13, 'B-Herstellungsland und -region': 14, 'B-Innensohlenmaterial': 15, 'B-Jahreszeit': 16, 'B-Marke': 17, 'B-Maßeinheit': 18, 'B-Modell': 19, 'B-Muster': 20, 'B-Obermaterial': 21, 'B-Produktart': 22, 'B-Produktlinie': 23, 'B-Schuhschaft-Typ': 24, 'B-Stil': 25, 'B-Thema': 26, 'B-UK-Schuhgröße': 27, 'B-US-Schuhgröße': 28, 'B-Verschluss': 29, 'B-Zwischensohlen-Typ': 30, 'I-Abteilung': 31, 'I-Aktivität': 32, 'I-Akzente': 33, 'I-Anlass': 34, 'I-Besonderheiten': 35, 'I-Charakter': 36, 'I-EU-Schuhgröße': 37, 'I-Erscheinungsjahr': 38, 'I-Farbe': 39, 'I-Herstellernummer': 40, 'I-Herstellungsland und -region': 41, 'I-Marke': 42, 'I-Modell': 43, 'I-Muster': 44, 'I-Obermaterial': 45, 'I-Produktart': 46, 'I-Produktlinie

In [ ]:
len(tag2idx)

121

In [ ]:
# Convert tags to indices
tag_value_list = [
    [tag2idx[tag] for tag in tags]
    for tags in tags_list
]

In [ ]:
# Sanity check
tag_value_list[4]

[105,
 23,
 77,
 19,
 73,
 85,
 97,
 107,
 85,
 97,
 85,
 10,
 64,
 85,
 8,
 62,
 85,
 28,
 82,
 85,
 85,
 85]

In [ ]:
# Save tag_values as it will be required for later use.
t_values = open("tag_values.pkl", "wb")
pickle.dump(tag_values, t_values)
t_values.close()

**Set-up BERT tokenizer from pre-trained model for the tokens**

Note that transformers are often pretrained with subword tokenizers (i.e. BERT uses WordPiece, so we have to make our tokens to similar format), meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer.
This means that we need to do some processing on our tags/labels as the input ids returned by the tokenizer are longer than the lists of labels our dataset contain.
This is happening, first because some special tokens might be added (we can see a [CLS] and a [SEP] per above example) and then because of those possible splits of words in multiple tokens.

The below function tokenize_and_align_labels does 2 jobs:

- Setting –100 as the label for these special tokens and the subwords we wish to mask during training:
Why did we choose –100 as the ID to mask subword representations? The reason is that in PyTorch the cross-entropy loss class torch.nn.CrossEntropyLoss has an attribute called ignore_index whose value is –100. This index is ignored during training,

- We can use it to ignore the tokens associated with consecutive subwords.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
def tokenize_and_align_labels(tokens, ner_tags, label_all_tokens=True):
    tokenized_inputs = tokenizer(tokens, truncation=True, is_split_into_words=True)
    label_ids = []
    word_ids = tokenized_inputs.word_ids()
    previous_word_idx = None

    for word_idx in word_ids:
        if word_idx is None:
        # set –100 as the label for these special tokens
            label_ids.append(-100)
            # label_ids.append(0)
        # For the other tokens in a word, we set the label to either the current label or -100, depending on
        # the label_all_tokens flag.
        elif word_idx != previous_word_idx:
        # if current word_idx is != prev then its the most regular case
        # and add the corresponding token
            label_ids.append(ner_tags[word_idx])
        else:
        # to take care of sub-words which have the same word_idx
        # set -100 as well for them, but only if label_all_tokens == False
            label_ids.append(ner_tags[word_idx] if label_all_tokens else -100)
            # mask the subword representations after the first subword
        previous_word_idx = word_idx
    tokenized_inputs["labels"] = label_ids
    return tokenized_inputs

Example showing how the above code work:

In [ ]:
# Test an example on the above code
print(tokens_list[2])

['♥', 'MICHAEL', 'KORS', 'Sneaker', 'Gr', '39', '♥']


In [ ]:
tokenized_input = tokenizer(tokens_list[2], is_split_into_words=True)
word_ids = tokenized_input.word_ids()
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)
print(word_ids)
print(len(tokens))
print(len(word_ids))

['[CLS]', '♥', 'MI', '##CH', '##A', '##EL', 'KO', '##RS', 'S', '##nea', '##ker', 'G', '##r', '39', '♥', '[SEP]']
[None, 0, 1, 1, 1, 1, 2, 2, 3, 3, 3, 4, 4, 5, 6, None]
16
16


In [ ]:
# Before applying the tokenize_and_align_labels function, the below result doesn't have labels data.
tokenized_input

{'input_ids': [101, 1871, 75416, 86448, 10738, 68999, 67422, 39469, 156, 32239, 11880, 144, 10129, 11303, 1871, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
# After executing tokenize_and_align_labels function, the labels info. is attached.
# Also we have ensured that the length of input_ids, word_ids, attention_mask
# and labels are all the same.
q = tokenize_and_align_labels(tokens_list[2], tag_value_list[2])

In [ ]:
print(q)
print(len(q['input_ids']))
print(len(q['attention_mask']))
print(len(q['labels']))

{'input_ids': [101, 1871, 75416, 86448, 10738, 68999, 67422, 39469, 156, 32239, 11880, 144, 10129, 11303, 1871, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 85, 17, 17, 17, 17, 71, 71, 114, 114, 114, 85, 85, 95, 85, -100]}
16
16
16


**Apply the tokenize_and_align_labels to the datasets (tokens_list and tag_value_list).**

In [ ]:
%%time
tokenized_datasets = [tokenize_and_align_labels(tokens, tags) for tokens, tags in zip(tokens_list, tag_value_list)]

CPU times: user 1.2 s, sys: 29.5 ms, total: 1.23 s
Wall time: 2.96 s


In [ ]:
#sanity check
print(tokenized_datasets[2])

{'input_ids': [101, 1871, 75416, 86448, 10738, 68999, 67422, 39469, 156, 32239, 11880, 144, 10129, 11303, 1871, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 85, 17, 17, 17, 17, 71, 71, 114, 114, 114, 85, 85, 95, 85, -100]}


In [ ]:
# Find the length of the longest 'input_ids' sequence (the longest tokenized tokens, to be used in the Max Len)
max_input_ids_length = max(len(item['input_ids']) for item in tokenized_datasets)

print("Length of the longest 'input_ids' sequence:", max_input_ids_length)

Length of the longest 'input_ids' sequence: 53


In [ ]:
# Save the tokenizer as we will also require tokenizer for later use.
save_tokenizer = open("tokenizer.pkl", "wb")
pickle.dump(tokenizer, save_tokenizer)
save_tokenizer.close()

**Prepare training and testing data**

In [ ]:
# Here, we split data into 90% training set and 10% testing set. We will use k-fold cross validation
# to further split the training set into 80% for training and 20% for validation in the training process.
# Therefore the ratios are approx. as below:
# train_ratio = 0.72
# val_ratio = 0.18
# test_ratio = 0.1

# tokenized_datasets is the DataFrame
train_data, test_data = train_test_split(tokenized_datasets, test_size=0.1, random_state=42)

# Resulting shapes of the datasets
print("Train set shape:", len(train_data))
print("Test set shape:", len(test_data))

Train set shape: 4500
Test set shape: 500


In [ ]:
#sanity check
print("train_data[0]:",train_data[0])
print("test_data[0]:",test_data[0])

train_data[0]: {'input_ids': [101, 32992, 95011, 24093, 39039, 13924, 10174, 80468, 30340, 11259, 15985, 39774, 93089, 11733, 91075, 14703, 11369, 42622, 47271, 11369, 103737, 145, 11281, 63809, 43567, 11373, 10729, 10350, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 105, 105, 105, 105, 19, 19, 73, 73, 73, 85, 23, 23, 23, 47, 47, 47, 47, 47, 47, 77, 112, 112, 97, 100, 100, 100, 100, -100]}
test_data[0]: {'input_ids': [101, 156, 10550, 70214, 36923, 35679, 11641, 24820, 38644, 55260, 13723, 11643, 156, 32239, 11880, 36750, 12044, 18593, 10112, 40317, 11373, 11211, 113, 39050, 118, 49622, 11733, 114, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

**Set up the metrics function**

**seqeval** - The way the package works by accepting list of lists
The seqeval package expects the predictions and labels as lists of lists, with each list corresponding to a single example in our validation or test sets. To integrate these metrics during training, we need a function that can take the outputs of the model and convert them into the lists that seqeval expects.

The following does the trick by ensuring we ignore the label IDs associated with subsequent subwords:

In [ ]:
# Load the metric
metric = datasets.load_metric("seqeval")

<ipython-input-49-2183900eedc5>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("seqeval")


Test a sample on the metric.compute() function.

In [ ]:
testsample = tag_value_list[2]
print(testsample)
validsample = tags_list[2]
print(validsample)

[85, 17, 71, 114, 85, 95, 85]
['NoTag', 'B-Marke', 'L-Marke', 'U-Stil', 'NoTag', 'U-EU-Schuhgröße', 'NoTag']


In [ ]:
testtag = [tag_values[i] for i in testsample]

metric.compute(predictions=[testtag], references=[validsample])

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NoTag seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: L-Marke seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: U-Stil seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: U-EU-Schuhgröße seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'EU-Schuhgröße': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'Marke': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'Stil': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'oTag': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [ ]:
def compute_metrics(eval_preds):
    """
    Function to compute the evaluation metrics for Named Entity Recognition (NER) tasks.
    The function computes precision, recall, F1 score and accuracy.

    Parameters:
    eval_preds (tuple): A tuple containing the predicted logits and the true labels.

    Returns:
    A dictionary containing the precision, recall, F1 score and accuracy.
    """
    pred_logits, labels = eval_preds

    pred_logits = np.argmax(pred_logits, axis=2)
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax

    # We remove all the values where the label is -100
    predictions = [
        [idx2tag[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
      [idx2tag[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(pred_logits, labels)
   ]

    results = metric.compute(predictions=predictions, references=true_labels)
    flattened_results = {
   "precision": results["overall_precision"],
   "recall": results["overall_recall"],
   "f1": results["overall_f1"],
  "accuracy": results["overall_accuracy"],
  }
    return flattened_results

# Pull and fine-tune the pretained model

In [ ]:
# Model: bert-base-multilingual-cased
model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(tag2idx), output_attentions=False, output_hidden_states=False)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Load in the saved state_dict()
#model.load_state_dict(torch.load("drive/My Drive/Deep Dream/BERT base - multilingual/Previous best/ner_model.pth"))

In [ ]:
# Load the previous trained model from saved path

#model = BertForTokenClassification.from_pretrained("drive/My Drive/Deep Dream/BERT base - multilingual/Current")

In [ ]:
#Make sure the transformer and accelerate version are update to dates
transformers.__version__, accelerate.__version__

('4.35.2', '0.24.1')

In [ ]:
model.cuda();

# Training and evaluation

In [ ]:
#Define training args

args = TrainingArguments(
"test-ner",
evaluation_strategy = "epoch",
learning_rate=1e-5, # 3e-4
per_device_train_batch_size=100,
per_device_eval_batch_size=100,
num_train_epochs=3,#
adam_beta1 = 0.9,
adam_beta2 = 0.999,
adam_epsilon = 1e-08,
weight_decay=0,
logging_strategy="epoch",
#save_strategy="epoch"  # Save model after each epoch
)

In [ ]:
# #Define training args

# args = TrainingArguments(
#   # output_dir="drive/My Drive/SCU COEN 240 Machine Learning/trainedModel", # output directory
#   "test-ner",
#   per_device_train_batch_size= 100, # batch size per device during training
#   evaluation_strategy="epoch",    # evaluation strategy to adopt during training
#   # Save strategy = "epoch"
#   learning_rate=3e-4,
#   # gradient_accumulation_steps = 4,
#   # optimizer = optimizer,
#   weight_decay = 0.0,
#   adam_beta1 = 0.9,
#   adam_beta2 = 0.999,
#   adam_epsilon = 1e-08,
  # max_grad_norm = 1.0,
#   num_train_epochs=3,             # total number of training epochs
#   # fp16=True,                    # use mixed precision
#   # eval_steps=1000,                 # number of update steps before evaluating
#   logging_strategy="epoch",           # number of update steps before logging
#   # save_strategy="no",  # Or set to "no" to disable saving checkpoints
#   # save_steps=0,  # Set to -1 or 0 to disable saving based on steps
#   # eval_steps=50,
#   # logging_steps=50,
#   # save_total_limit=2,             # limit the total amount of checkpoints on disk
#   remove_unused_columns=False,    # remove unused columns from the dataset
#   # push_to_hub=False,              # do not push the model to the hub
#   report_to='tensorboard',        # report metrics to tensorboard
#   # load_best_model_at_end=True,    # load the best model at the end of training
# )

In [ ]:
# Load the data collator
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
# Use KFold to train the model

from sklearn.model_selection import KFold

# Define k-fold cross-validation: set n_splits=10 so we can 80% for training and 20% for validation.
k_folds = 5 #
skf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
fold_accuracies = []

# data = [item['input_ids'] for item in train_data]
# label = [item['labels'] for item in train_data]

for fold, (train_indices, val_indices) in enumerate(skf.split(train_data)):
    print(f"Training Fold {fold+1}/{k_folds}")

    # Split data into training and validation sets for this fold
    train_set = [train_data[idx] for idx in train_indices]
    val_set = [train_data[idx] for idx in val_indices]

    optimizer = AdamW(model.parameters(), lr=args.learning_rate)
    criterion = nn.CrossEntropyLoss()

    # Calculate total steps for the scheduler
    total_steps = len(train_set) * args.num_train_epochs

    # Create the scheduler
    scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
    )

    args.scheduler = scheduler  # Set the scheduler in the training arguments

    # Initialize Trainer for this fold
    trainer = Trainer(
      model,
      args,
    train_dataset=train_set,
    eval_dataset=val_set,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    #optimizers=optimizer
    )

    # Train the model for this fold
    trainer.train()

#     fold_accuracy = compute_metrics["f1"]
#     fold_accuracies.append(fold_accuracy)
#     print(f"Accuracy for Fold {fold+1}: {fold_accuracy}")

# # Calculate average accuracy across all folds
# average_accuracy = sum(fold_accuracies) / len(fold_accuracies)
# print(f"Average Accuracy: {average_accuracy}")

**Save the Model for later use.**

In [ ]:
# Define model save path

MODEL_SAVE_PATH = "drive/My Drive/Deep Dream/BERT base - multilingual/Current/ner_model.pth"

# Save the model state dict
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model.state_dict(), # saving the state_dict() only saves the learned parameters
           f=MODEL_SAVE_PATH)

Saving model to: drive/My Drive/Deep Dream/BERT base - multilingual/Current/ner_model.pth


# Loading model & prediction

In [ ]:
# # Define The Model Class
ner_model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(tag2idx), output_attentions=False, output_hidden_states=False)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Load in the saved state_dict()
ner_model.load_state_dict(torch.load("drive/My Drive/Deep Dream/BERT base - multilingual/Current/ner_model.pth"))

<All keys matched successfully>

**Compute f1 score for trained NER model**

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cuda


In [ ]:
# Predict the testing set and evaluate the model performance.

def predict(ner_model, testing_set, device):
    model = ner_model.to(device)
    predictions_list = []
    labels_list  = []
    model.eval()

    for item in testing_set:
        input_ids = item['input_ids']
        token_type_ids = item['token_type_ids']
        attention_mask = item['attention_mask']
        labels = item['labels']

        with torch.no_grad():  # Ensure no gradient calculations during inference
            input_ids = torch.tensor([input_ids]).to(device)
            token_type_ids = torch.tensor([token_type_ids]).to(device)
            attention_mask = torch.tensor([attention_mask]).to(device)
            outputs = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
            predicted_labels = torch.argmax(outputs.logits, axis=-1)
            predictions_list.append(predicted_labels.tolist()[0])
            labels_list.append(labels)


    # Convert predicted label indices to label words using idx2tag dictionary
    predicted_label_words = [
            [idx2tag[pred] for (pred, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions_list, labels_list)
        ]

    actual_label_words = [
          [idx2tag[l] for (pred, l) in zip(prediction, label) if l != -100]
          for prediction, label in zip(predictions_list, labels_list)
      ]

    # Calculate precision, recall, and F1-score
    flat_predicted = [label for sublist in predicted_label_words for label in sublist]
    flat_actual = [label for sublist in actual_label_words for label in sublist]

    # Calculate precision, recall, and F1-score
    precision, recall, f1_score, _ = precision_recall_fscore_support(flat_actual, flat_predicted, average='weighted')

    # Print overall precision, recall, and F1-score
    print(f"Overall Precision: {precision:.4f}")
    print(f"Overall Recall: {recall:.4f}")
    print(f"Overall F1-score: {f1_score:.4f}")

    all_class = metric.compute(predictions=predicted_label_words, references=actual_label_words)
    print(all_class)


In [ ]:
# Perform prediction
ner_model = model
testing_set = test_data

# predict(bert1_model, testing_set, device)
predict(ner_model, testing_set, device)

In [ ]:
All_class=[{'Abteilung': {'precision': 0.98, 'recall': 0.9884726224783862, 'f1': 0.9842180774748924, 'number': 347}, 'Aktivität': {'precision': 0.8974358974358975, 'recall': 0.8860759493670886, 'f1': 0.8917197452229298, 'number': 79}, 'Akzente': {'precision': 0.875, 'recall': 0.6363636363636364, 'f1': 0.7368421052631579, 'number': 11}, 'Anlass': {'precision': 0.9558823529411765, 'recall': 0.9701492537313433, 'f1': 0.962962962962963, 'number': 67}, 'Besonderheiten': {'precision': 0.6, 'recall': 0.6, 'f1': 0.6, 'number': 40}, 'Charakter': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}, 'Charakter Familie': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}, 'Dämpfungsgrad': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1}, 'EU-Schuhgröße': {'precision': 0.8366336633663366, 'recall': 0.8325123152709359, 'f1': 0.8345679012345679, 'number': 203}, 'Erscheinungsjahr': {'precision': 1.0, 'recall': 0.8333333333333334, 'f1': 0.9090909090909091, 'number': 6}, 'Farbe': {'precision': 0.8454810495626822, 'recall': 0.8868501529051988, 'f1': 0.8656716417910448, 'number': 327}, 'Gewebeart': {'precision': 0.65, 'recall': 0.9285714285714286, 'f1': 0.7647058823529412, 'number': 14}, 'Herstellernummer': {'precision': 0.90625, 'recall': 0.9273255813953488, 'f1': 0.9166666666666666, 'number': 344}, 'Herstellungsland und -region': {'precision': 0.5, 'recall': 0.5, 'f1': 0.5, 'number': 6}, 'Innensohlenmaterial': {'precision': 0.6, 'recall': 1.0, 'f1': 0.7499999999999999, 'number': 3}, 'Jahreszeit': {'precision': 0.7142857142857143, 'recall': 1.0, 'f1': 0.8333333333333333, 'number': 5}, 'Laufsohlenmaterial': {'precision': 1.0, 'recall': 0.6666666666666666, 'f1': 0.8, 'number': 3}, 'Marke': {'precision': 0.9373776908023483, 'recall': 0.9247104247104247, 'f1': 0.9310009718172984, 'number': 518}, 'Maßeinheit': {'precision': 1.0, 'recall': 0.8888888888888888, 'f1': 0.9411764705882353, 'number': 9}, 'Modell': {'precision': 0.7503875968992249, 'recall': 0.7311178247734139, 'f1': 0.7406273909716908, 'number': 662}, 'Muster': {'precision': 0.18181818181818182, 'recall': 0.2222222222222222, 'f1': 0.19999999999999998, 'number': 9}, 'Obermaterial': {'precision': 0.9134615384615384, 'recall': 0.9313725490196079, 'f1': 0.9223300970873786, 'number': 102}, 'Produktart': {'precision': 0.9683195592286501, 'recall': 0.9656593406593407, 'f1': 0.9669876203576341, 'number': 728}, 'Produktlinie': {'precision': 0.7647058823529411, 'recall': 0.826271186440678, 'f1': 0.7942973523421588, 'number': 236}, 'Schuhschaft-Typ': {'precision': 0.9852941176470589, 'recall': 0.9436619718309859, 'f1': 0.9640287769784172, 'number': 71}, 'Schuhweite': {'precision': 0.5, 'recall': 1.0, 'f1': 0.6666666666666666, 'number': 1}, 'Stil': {'precision': 0.9840764331210191, 'recall': 0.9716981132075472, 'f1': 0.9778481012658228, 'number': 318}, 'Thema': {'precision': 0.7777777777777778, 'recall': 0.7777777777777778, 'f1': 0.7777777777777778, 'number': 27}, 'UK-Schuhgröße': {'precision': 0.7222222222222222, 'recall': 0.8666666666666667, 'f1': 0.7878787878787877, 'number': 30}, 'US-Schuhgröße': {'precision': 0.9428571428571428, 'recall': 0.8918918918918919, 'f1': 0.9166666666666667, 'number': 37}, 'Verschluss': {'precision': 0.9722222222222222, 'recall': 0.9459459459459459, 'f1': 0.9589041095890412, 'number': 37}, 'Zwischensohlen-Typ': {'precision': 0.8333333333333334, 'recall': 1.0, 'f1': 0.9090909090909091, 'number': 5}, 'oTag': {'precision': 0.8268156424581006, 'recall': 0.8457142857142858, 'f1': 0.8361581920903954, 'number': 700}}]

In [ ]:
# Codes to visulize the f1 for each class

aspect_names = []
f1_values = []
numbers = []

# Extract data from the list
for item in All_class:
    for aspect_name, values in item.items():
        if isinstance(values, dict):  # Check if the value is a dictionary
            aspect_names.append(aspect_name)
            f1_values.append(values.get('f1', None))  # Use .get() to handle missing keys
            numbers.append(values.get('number', None))

# Create a DataFrame
data = {
    'Aspect Names': aspect_names,
    'f1': f1_values,
    'number': numbers
}

class_perf = pd.DataFrame(data)

# Sort by 'f1' in ascending order
class_perf = class_perf.sort_values('f1', ascending=True)
class_perf


,Aspect Names,f1,number
5,Charakter,0.000000,2
6,Charakter Familie,0.000000,1
20,Muster,0.200000,9
13,Herstellungsland und -region,0.500000,6
4,Besonderheiten,0.600000,40
25,Schuhweite,0.666667,1
2,Akzente,0.736842,11
19,Modell,0.740627,662
14,Innensohlenmaterial,0.750000,3
11,Gewebeart,0.764706,14


In [ ]:
class_perf.shape

(33, 3)

In [ ]:
o = 'oTag'
uu = o.split('-')
uu

['oTag']

# Output the prediction on the format as required

In [ ]:
# Output the result as per required.

def output(ner_model, input_data, device):
    model = ner_model.to(device)
    output_data = []
    model.eval()

    for index, title in input_data.iterrows():
        rec_num = title['Record Number']
        title_to_word = title['Title'].split()
        # print("title_to_word is", title_to_word)
        title_to_token = tokenizer(title_to_word, is_split_into_words=True)
        word_ids = title_to_token.word_ids()
        input_ids = title_to_token['input_ids']
        token_type_ids = title_to_token['token_type_ids']
        attention_mask = title_to_token['attention_mask']

        with torch.no_grad():  # Ensure no gradient calculations during inference
            input_ids = torch.tensor([input_ids]).to(device)
            token_type_ids = torch.tensor([token_type_ids]).to(device)
            attention_mask = torch.tensor([attention_mask]).to(device)
            outputs = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
            predicted_labels = torch.argmax(outputs.logits, axis=-1)
            predictions_list = predicted_labels.tolist()[0]

        # print("the pred list is", predictions_list)
        # Extract the actual predicted tags per word based on unique word_ids
        # and convert predicted label indices to label words using idx2tag dictionary
        pre_wordid = None
        Pre_realTag = None
        predicted_tag = []

        for i in range(1, len(predictions_list)-1):
            if word_ids[i] != pre_wordid:
                predicted_tag.append(idx2tag[predictions_list[i]])
            pre_wordid = word_ids[i]

        # print("the pred tag is", predicted_tag)
        # Match the tag with word / words, also need to consider combing the words together.
        word = ''
        special = ['.', ':', ',', '-', "'", '_', '/', '❤', '#', '~', '&', ';', '!', '♀', '♡', '•', '·', '+', '"']
        for i in range(len(predicted_tag)):
            tempTag = predicted_tag[i].split('-')
            if len(tempTag) > 1:
                realTag = '-'.join(tempTag[1:])
                if tempTag[0] == 'U' and realTag == 'EU-Schuhgröße':
                    if title_to_word[i].isdigit() and int(title_to_word[i]) < 20:
                        realTag = 'US-Schuhgröße'

            if tempTag[0] in ['oTag', 'No Tag', 'Obscure'] and len(word) == 0:
                continue

            elif tempTag[0] in ['oTag', 'No Tag', 'Obscure']:
                output_data.append([rec_num, Pre_realTag, word])
                word = ''
                continue

            elif tempTag[0] == 'U' and title_to_word[i] in special:
                if len(word) != 0:
                  output_data.append([rec_num, Pre_realTag, word])
                  word = ''
                continue

            # below might need to be comment out
            elif tempTag[0] == 'B' and i< len(predicted_tag)-1 and predicted_tag[i+1].split('-')[0] not in ['I', 'L']:
                  if len(word) != 0:
                    output_data.append([rec_num, Pre_realTag, word])
                    word = ''
                  continue

            elif tempTag[0] == 'B' and i == len(predicted_tag)-1:
                  if len(word) != 0:
                    output_data.append([rec_num, Pre_realTag, word])
                    word = ''
                  continue
            # above might need to be comment out

            elif (tempTag[0] == 'U' or tempTag[0] == 'B') and len(word) != 0:
                output_data.append([rec_num, Pre_realTag, word])
                word = ''
                Pre_realTag = realTag
                word += title_to_word[i]
                continue

            elif (tempTag[0] == 'U' or tempTag[0] == 'B') and len(word) == 0:
                Pre_realTag = realTag
                word += title_to_word[i]
                continue

            elif tempTag[0] == 'I' or tempTag[0] == 'L':
                Pre_realTag = realTag
                word += " " + title_to_word[i]
                continue

            # elif tempTag[0] == 'O':
            #     output_data.append([rec_num, Pre_realTag, word])
            #     word = ''
            #     continue

            # elif tempTag[0] == 'B' and len(word) != 0:
            #     output_data.append([rec_num, Pre_realTag, word])
            #     word = ''
            #     Pre_realTag = realTag
            #     word += title_to_word[i]
            #     continue

            # elif tempTag[0] == 'B' and len(word) == 0:
            #     Pre_realTag = realTag
            #     word += " " + title_to_word[i]
            #     continue

            # elif tempTag[0] == 'I' and realTag == Pre_realTag:
            #     Pre_realTag = realTag
            #     word += " " + title_to_word[i]
            #     continue

            # elif tempTag[0] == 'I' and len(word) != 0 and realTag != Pre_realTag:
            #     output_data.append([rec_num, Pre_realTag, word])
            #     word = ''
            #     Pre_realTag = realTag
            #     word += title_to_word[i]

        if len(word) != 0:
            output_data.append([rec_num, Pre_realTag, word])

    output_df = pd.DataFrame(output_data)
    return output_df


In [ ]:
special = ['.', ':', ',', '-', "'", '_', '/', '❤', '#', '~', '&', ';', '!', '♀', '♡', '•', '·', '+', '"']
if '-' in special:
  print("yes")

yes


Test an example to show the output

In [ ]:
exam = listing_titles.iloc[14754:14755]
exam

,Record Number,Title
14754,14755,Damen Converse Chucks hoch pink Gr . 40 / 7 su...


In [ ]:
# Testing an example to show the output

testing = df[:2]
testing = testing.iloc[:, :2]
print(testing)

   Record Number                                              Title
0              1  Supreme Nike SB Dunk High By any Means Red US1...
1              2  New Balance 530 Männer und Frauen Laufschuhe m...


In [ ]:
len(['Sneaker', 'in', 'Gr', '.', '3', '1/2', 'oder', '36', 'Weite', 'G', 'von', 'CAPRICE', 'aus', 'Echtleder', '##'])

15

In [ ]:
len(['B-Stil', 'O', 'O', 'B-EU-Schuhgröße', 'B-UK-Schuhgröße', 'I-UK-Schuhgröße', 'O', 'B-EU-Schuhgröße', 'O', 'B-Schuhweite', 'O', 'B-Marke', 'O', 'B-Obermaterial', 'O'])

15

In [ ]:
# Output the prediction on the format as required
input_data = exam
#ner_model = model

print("The Output prediction is:")
output(ner_model, input_data, device)

The Output prediction is:


,0,1,2
0,14755,Abteilung,Damen
1,14755,Marke,Converse
2,14755,Produktlinie,Chucks
3,14755,Farbe,pink
4,14755,EU-Schuhgröße,40
5,14755,UK-Schuhgröße,7
6,14755,Produktlinie,All Star


In [ ]:
sampletesting = check[check[0] == 5060]
sampletesting

,0,1,2
396,5060,Stil,Sneaker
397,5060,EU-Schuhgröße,.
398,5060,UK-Schuhgröße,3 1/2
399,5060,EU-Schuhgröße,36
400,5060,Schuhweite,G
401,5060,Marke,CAPRICE
402,5060,Obermaterial,Echtleder


# Predict the Quiz set

In [ ]:
# Load the full dataset

listing_path = "drive/My Drive/Deep Dream/Listing_Titles.tsv"
# Read the file using Python's open to handle line terminators
with open(listing_path, 'r', encoding='utf-8', newline='\r\n') as file:
    listing_titles = pd.read_csv(file, sep='\t', quotechar='', quoting=3)

In [ ]:
# Show the full dataframe
listing_titles.tail()

,Record Number,Title
9999995,9999996,Skechers Compulsions Z Strap Damen Schwarz Tur...
9999996,9999997,Neu bogner 199 . Euro neupreis
9999997,9999998,New Balance M670 Herren US 11 Schwarz Tennissc...
9999998,9999999,Pepe Jeans Tinker Pro 73 Edt 20 Sneakers Herre...
9999999,10000000,HERREN SCHUHE SNEAKERS ADIDAS ORIGINALS X_PLR ...


In [ ]:
# Extract the Quiz dataset
# The Quiz Data consists of records 5001 to 30000 of the listing data, inclusively.

Quiz_data = listing_titles[5000:30000]

In [ ]:
# Ensure the extract quiz data is correct

Quiz_data.shape[0]

25000

In [ ]:
Quiz_data.tail()

,Record Number,Title
29995,29996,Herrenschuhe Ecco EUR 42 Sportschuhe Sneaker F...
29996,29997,ASICS Damen Novablast Road Running Laufschuhe ...
29997,29998,Nike Dunk Low Grey Fog | verschiedene Größen |...
29998,29999,PUMA Karmen Rebelle Damen Weiss F01
29999,30000,Puma Turnschuhe weiß unisex 43 wie neu


In [ ]:
# Output the prediction on the format as required
input_data = Quiz_data
#ner_model = model

print("The Output prediction is:")
output_data = output(ner_model, input_data, device)
output_data.tail()

The Output prediction is:


,0,1,2
152234,30000,Marke,Puma
152235,30000,Produktart,Turnschuhe
152236,30000,Farbe,weiß
152237,30000,Abteilung,unisex
152238,30000,EU-Schuhgröße,43


Export the output data

In [ ]:
# Export the output to a utf-8 encoded tsv (TAB-separated) file

file_path = '/content/drive/My Drive/Deep Dream/BERT base - multilingual/output_data_bert-base-multilingual.tsv'
output_data.to_csv(file_path, sep='\t', encoding='utf-8', index=False, header=False)

In [ ]:
# check
# file_path = '/content/drive/My Drive/Deep Dream/BERT base - multilingual/output_data_bert-base-multilingual.tsv'
file_path = '/content/drive/My Drive/Deep Dream/BERT base - multilingual/output_data_bert-base-multilingual.tsv'

# Read the TSV file into a DataFrame
check = pd.read_csv(file_path, sep='\t', header=None)
check.tail()

,0,1,2
152234,30000,Marke,Puma
152235,30000,Produktart,Turnschuhe
152236,30000,Farbe,weiß
152237,30000,Abteilung,unisex
152238,30000,EU-Schuhgröße,43


In [ ]:
# Check for missing or NaN values in the aspect name column (the second column (index 1))
missing_values = check.iloc[:, 1].isnull().sum()

if missing_values > 0:
    print(f'There are {missing_values} missing or NaN values in the aspect name column.')
    # Display rows with missing or NaN values in the aspect name column
    print(check[check.iloc[:, 1].isnull()])
else:
    print('No missing or NaN values found in the aspect name column.')

No missing or NaN values found in the aspect name column.


In [ ]:
# Get rows with any missing or NaN values
rows_with_missing_values = check[check.iloc[:, 1].isnull()]

# Print rows with missing or NaN values
print("Rows with missing or NaN values:")
rows_with_missing_values

Rows with missing or NaN values:


,0,1,2


In [ ]:
# FULL_FINETUNING = True
# if FULL_FINETUNING:
#     param_optimizer = list(model.named_parameters())
#     no_decay = ['bias', 'gamma', 'beta']
#     optimizer_grouped_parameters = [
#         {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},
#         {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}
#     ]
# else:
#     param_optimizer = list(model.classifier.named_parameters)
#     optimizer_grouped_parameters = [{'params': [p for n, p in param_optimizer]}]

In [ ]:
# # Set up the hyperparameters

# learning_rate=3e-4
# EPOCHS = 3
# BATCH_SIZE = 100
# MAX_GRAD_NORM = 1.0
# MAX_LEN = 60
# optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=1e-8)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# def tensor_dataset(dataset):
#     input_ids = [item['input_ids'] for item in dataset]
#     #token_type_ids = [item['token_type_ids'] for item in dataset]
#     attention_mask = [item['attention_mask'] for item in dataset]
#     labels = [item['labels'] for item in dataset]

#     input_ids = pad_sequences([id for id in input_ids], maxlen=MAX_LEN, dtype='long', value=0.0, truncating='post', padding='post')
#     #token_type_ids = pad_sequences([i for i in token_type_ids], maxlen=MAX_LEN, value=tag2idx['PAD'], padding='post', dtype='long', truncating='post')
#     attention_mask = [[float(i != 0.0) for i in ii] for ii in input_ids]
#     labels = pad_sequences([l for l in labels], maxlen=MAX_LEN, value=tag2idx['PAD'], padding='post', dtype='long', truncating='post')

#     return TensorDataset(torch.tensor(input_ids), torch.tensor(attention_mask), torch.tensor(labels))

In [ ]:
# # Train the model

# from sklearn.model_selection import KFold
# #%%time

# # Define k-fold cross-validation: set n_splits=5 so we can 80% for training and 20% for validation.
# k_folds = 5
# skf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
# num_labels=len(tag2idx)

# #Define the loss function
# criterion = nn.CrossEntropyLoss()

# for fold, (train_indices, val_indices) in enumerate(skf.split(train_data)):
#     print(f"Training Fold {fold+1}/{k_folds}")
#     fold_F1s = []

#     # Split data into training and validation sets for this fold
#     train_set = [train_data[idx] for idx in train_indices]
#     val_set = [train_data[idx] for idx in val_indices]

#     # Create data-loaders.

#     tensor_train_set = tensor_dataset(train_set)
#     train_sampler = RandomSampler(tensor_train_set)
#     train_dataloader = DataLoader(tensor_train_set, sampler=train_sampler, batch_size=BATCH_SIZE)

#     tensor_val_set = tensor_dataset(val_set)
#     val_sampler = SequentialSampler(tensor_val_set)
#     val_dataloader = DataLoader(tensor_val_set, sampler=val_sampler, batch_size=BATCH_SIZE)

#     # Initiate the scheduler
#     total_steps = len(train_dataloader) * EPOCHS
#     scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

#     loss_values, validation_loss_values = [], []

#     for e in range(EPOCHS):
#         print(f'- Epoch 0{e+1} -')
#         model.train()
#         total_loss = 0

#         for step, batch in enumerate(train_dataloader):
#             batch = tuple(t.to(device) for t in batch)
#             #batch = tuple(t for t in batch)

#             #b_input_ids, b_type_ids, b_input_mask, b_labels = batch
#             b_input_ids, b_input_mask, b_labels = batch
#             model.zero_grad()
#             #outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
#             outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
#             #loss = outputs[0]
#             loss = criterion(outputs.logits.view(-1, num_labels), b_labels.view(-1))
#             loss.backward()
#             total_loss += loss.item()
#             torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=MAX_GRAD_NORM)
#             optimizer.step()
#             scheduler.step()
#         avg_train_loss = total_loss / len(train_dataloader)
#         print('Average train loss:\t{:.5f}'.format(avg_train_loss))
#         loss_values.append(avg_train_loss)


#         model.eval()
#         eval_loss, eval_accuracy = 0, 0
#         predictions, true_labels = [], []

#         for batch in val_dataloader:
#             batch = tuple(t.to(device) for t in batch)
#             #batch = tuple(t for t in batch)
#             b_input_ids, b_input_mask, b_labels = batch

#             with torch.no_grad():
#                 #outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
#                 outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)

#             logits = outputs[1].detach().cpu().numpy()

#             label_ids = b_labels.to('cpu').numpy()
#             #loss_eval = outputs[0]
#             loss_eval = criterion(outputs.logits.view(-1, num_labels), b_labels.view(-1))
#             eval_loss += loss_eval.mean().item()
#             predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
#             true_labels.extend(label_ids)

#         eval_loss = eval_loss / len(val_dataloader)
#         validation_loss_values.append(eval_loss)
#         print('Validation loss:\t{:.5f}'.format(eval_loss))

#         # Convert predicted label indices to label words using idx2tag dictionary
#         predicted_label_words = [
#                 [idx2tag[pred] for (pred, l) in zip(prediction, label) if (l != -100 and idx2tag[l] != 'PAD')]
#                 for prediction, label in zip(predictions, true_labels)
#             ]

#         actual_label_words = [
#               [idx2tag[l] for (pred, l) in zip(prediction, label) if (l != -100 and idx2tag[l] != 'PAD')]
#               for prediction, label in zip(predictions, true_labels)
#           ]

#         results = metric.compute(predictions=predicted_label_words, references=actual_label_words)
#         flattened_results = {
#           "precision": "{:.5f}".format(results["overall_precision"]),
#           "recall": "{:.5f}".format(results["overall_recall"]),
#           "f1": "{:.5f}".format(results["overall_f1"]),
#           "accuracy": "{:.5f}".format(results["overall_accuracy"]),
#           }
#         fold_F1 = results["overall_f1"]
#         fold_F1s.append(fold_F1)
#         #print(f"Accuracy for Fold {fold+1}: {fold_accuracy}")
#         print(flattened_results)
#         print(" ")

#     # Calculate average accuracy for current fold
#     average_F1 = sum(fold_F1s) / len(fold_F1s)
#     print("Average F1: {:.5f}".format(average_F1))
#     print(" ")
